In [1]:
import os
import numpy as np
import pandas as pd
import shutil
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import csv

# puts contents from train.txt in a pandas dataframe to sort the training and validation datasets to folders from 0-22
data = pd.read_csv('../input/uos-com2028/train.txt', delimiter = " ", header=None)
# 80% training, 20% validation
split_size = int(len(data.index)*0.2)

# creates folders for validation and training
if not os.path.exists("/kaggle/working/validation/"):
    os.mkdir("/kaggle/working/validation/")
if not os.path.exists("/kaggle/working/training"):
    os.mkdir("/kaggle/working/training")

# sorts images into validation and training based off the train.txt dataframe to sort everything
for path, label in data.values[:split_size]:
    validation_path = "/kaggle/working/validation/" + str(label)
    if not os.path.exists(validation_path):
        os.mkdir(validation_path)
    shutil.copyfile("../input/uos-com2028/train/" + path, validation_path + path.strip("train"))
for path, label in data.values[split_size:]:
    training_path = "/kaggle/working/training/" + str(label)
    if not os.path.exists(training_path):
        os.mkdir(training_path)
    shutil.copyfile("../input/uos-com2028/train/" + path, training_path + path.strip("train"))

In [2]:
image_h = 150
image_w = 150
batch_size = 32

# makes this generalise images (data augmentation); applies this to all of the datasets (training, validation, testing)
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    brightness_range=[0.8,1.2],
    horizontal_flip=True
)

# image data generator for training dataset, resizes everything to (150, 150), makes it have a class size of 23, and shuffles images
train_generator = datagen.flow_from_directory(
    "/kaggle/working/training",
    target_size=(image_h, image_w),
    batch_size=batch_size,
    classes= [str(i) for i in range(0,23)],
    class_mode="categorical",
    shuffle=True
)

# same as the train_generator
validation_generator = datagen.flow_from_directory(
    "/kaggle/working/validation",
    target_size=(image_h, image_w),
    batch_size=batch_size,
    classes= [str(i) for i in range(0,23)],
    class_mode="categorical",
    shuffle=True
)

# slightly different to the other generators, shuffle is true by default, for a proper csv file it must be false
test_generator = datagen.flow_from_directory(
    '/kaggle/input/uos-com2028/test',
    target_size=(image_h, image_w),
    batch_size=32,
    shuffle=False,
    class_mode=None
)

# get lengths of the datasets for steps
train_n = len(train_generator.filenames)
val_n = len(validation_generator.filenames)
test_n = len(test_generator.filenames)
# (150, 150, 3) - 150 for shorter epoch times
input_shape = (image_h, image_w, 3)

Found 8216 images belonging to 23 classes.
Found 2054 images belonging to 23 classes.
Found 15009 images belonging to 1 classes.


In [4]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=input_shape,
    include_top=False)  # Do not include the ImageNet classifier at the top.

# high patience, and monitoring val_loss
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

base_model.trainable = False

inputs = keras.Input(shape=input_shape)
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (Categorical classification)

outputs = keras.layers.Dense(23)(x)
model = keras.Model(inputs, outputs)

# high learning rate for the base model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1, epsilon=0.1),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])
model.fit(
    train_generator,
    steps_per_epoch=train_n // batch_size,
    epochs=50,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=val_n // batch_size,
    callbacks=[callback]
)

base_model.trainable = True

# low learning rate for the new model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4, epsilon=1e-4),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])
model.fit(
    train_generator,
    steps_per_epoch=train_n // batch_size,
    epochs=50,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=val_n // batch_size,
    callbacks=[callback]
)

# base model average epoch times: 97s
# new model average epoch times: 113s

Epoch 1/50
256/256 [==============================] - 99s 380ms/step - loss: 2.1438 - categorical_accuracy: 0.4472 - val_loss: 2.2037 - val_categorical_accuracy: 0.5425
Epoch 2/50
256/256 [==============================] - 96s 375ms/step - loss: 2.2605 - categorical_accuracy: 0.5610 - val_loss: 3.0177 - val_categorical_accuracy: 0.5688
Epoch 3/50
256/256 [==============================] - 97s 379ms/step - loss: 2.5938 - categorical_accuracy: 0.6049 - val_loss: 3.4189 - val_categorical_accuracy: 0.5654
Epoch 4/50
256/256 [==============================] - 97s 378ms/step - loss: 2.4561 - categorical_accuracy: 0.6376 - val_loss: 3.8053 - val_categorical_accuracy: 0.5664
Epoch 5/50
256/256 [==============================] - 97s 379ms/step - loss: 2.6546 - categorical_accuracy: 0.6397 - val_loss: 3.8718 - val_categorical_accuracy: 0.5469
Epoch 6/50
256/256 [==============================] - 97s 378ms/step - loss: 2.9448 - categorical_accuracy: 0.6400 - val_loss: 4.2855 - val_categorical_acc

In [ ]:
predictions = model.predict(test_generator, steps=test_n // 32)

In [ ]:
row_list = [["id", "label"]]
for x in range(len(predictions)):
    row_list.append([test_generator.filenames[x].strip('test/.jpg'), np.argmax(predictions[x])])

with open("6586214.csv", "w", newline = "") as file:
    writer = csv.writer(file)
    writer.writerows(row_list)

row_list = None